## Making MDPs in Python

For the simple environments, we can just hardcode the MDPs into a dictionary by exhaustively encoding the whole state space and the transition function. We will also go through a more complicated example where the state space is too large to be manually coded and we need to implement the transition function based on some state parameters.

You can start this assignment during/after reading Grokking Ch-2.

NOTE: all page numbers referred to in this ipynb notebook refer to Grokking textbook

## Environment 0 - Bandit Walk

Let us consider the BW environment on Page 39. 

State Space has 3 elements, states 0, 1 and 2.
States 0 and 2 are terminal states and state 1 is the starting state.

Action space has 2 elements, left and right.

The environment is deterministic - transition probability of any action is 1.

Only 1 (State, Action, State') tuple has positive reward, (1, Right, 2) gives the agent +1 reward.

We'll model this MDP as a dictionary. This code is an example for the upcoming exercises.

In [32]:
bw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)]
    },

    1 : {
        "Right" : [(1, 2, 1, True)],
        "Left" : [(1, 0, 0, True)]
    },

    2 : {
        "Right" : [(1, 2, 0, True)],
        "Left" : [(1, 2, 0, True)]
    }
    
}

Here we a dictionary bw_mdp which to get the transitions possible we can do bw_mdp\[current state\]\[action to take\]. this will return a list of the possibilities.

The format of each element within the list is   
(probability of that transition , next state , reward , 'is the next state a terminal state?')

Obviously make sure that the sum of probability of all possible transitions is 1

Note that by convention, all actions from terminal states still lead to the same state with reward 0.

## Environment 1 - Slippery Walk

Here, states 0 and 6 are terminal states and state 3 is the starting state.

Action space has again 2 elements, left and right.

The environment is now stochastic, transition probability of any action is as follows -
If agent chooses `Right` at a non-terminal state,
- $50\%$ times it will go to `Right` state
- $33\frac{1}{3} \%$ times it will stay in same state
- $16\frac{2}{3}\%$ times it will go to `Left`state

And similiarly vice versa for Left(just replace left with right and right with left in previous list)

This time, 2 different (State, Action, State') tuples have positive rewards, you need to find them.

We'll again model this MDP as a dictionary. Part of the code is written for you.

In [33]:
swf_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)],
    },

    1 : {
        "Right" : [
            (1/2, 2, 0, False),
            (1/3, 1, 0, False),
            (1/6, 0, 0, True),
        ],
        "Left" : [
            (1/2, 0, 0, True),
            (1/3, 1, 0, False),
            (1/6, 2, 0, False),
        ],
    }, 

    2 : {
        "Right" : [
            (1/2, 3, 0, False),
            (1/3, 2, 0, False),
            (1/6, 1, 0, False),
        ],
        "Left" : [
            (1/2, 1, 0, False),
            (1/3, 2, 0, False),
            (1/6, 3, 0, False),
        ],
    },
    3 : {
        "Right" : [
            (1/2, 4, 0, False),
            (1/3, 3, 0, False),
            (1/6, 2, 0, False),
        ],
        "Left" : [
            (1/2, 2, 0, False),
            (1/3, 3, 0, False),
            (1/6, 4, 0, False),
        ],
    },
    4 : {
        "Right" : [
            (1/2, 5, 0, False),
            (1/3, 4, 0, False),
            (1/6, 3, 0, False),
        ],
        "Left" : [
            (1/2, 3, 0, False),
            (1/3, 4, 0, False),
            (1/6, 5, 0, False),
        ],
    },
    5 : {
        "Right" : [
            (1/2, 6, 1, True),
            (1/3, 5, 0, False),
            (1/6, 4, 0, False),
        ],
        "Left" : [
            (1/2, 4, 0, False),
            (1/3, 5, 0, False),
            (1/6, 6, 1, True),
        ],
    },
    6 : {
        "Right" : [(1, 6, 0, True)],
        "Left" : [(1, 6, 0, True)],
    }
    
}

## Environment 2 - Frozen Lake Environment

This environment is described on Page 46.

The FL environment has a large state space, so it's better to generate most of the MDP via Python instead of typing stuff manually.

Note that all 5 states - 5, 7, 11, 12, 15 are terminal states, so keep that in mind while constructing the MDP.

There are 4 actions now - Up, Down, Left, Right.

The environment is stochastic, and states at the border of lake will require separate treatment.



Yet again we will model this MDP as a (large) dictionary.

In [34]:
fl_mdp = {
    i : None for i in range(0, 16)
}

In [43]:
# Zoom Out to see the whole code so that it won't be that hard (Sorry it's my mistake)
for state in range(0, 16):

    # add transitions to other states
    transitions = {}

    for action in ["Up", "Down", "Right", "Left"]:
        
        transitions[action] = []
        if state not in [5, 7, 11, 12, 15]:
            if action != "Up" :
                transitions[action].append((1/3, state if state in [12,13,14,15] else state+4, int(state+4 == 15 and state not in [12,13,14,15]), bool(state+4 in [5,7,11,12,15] and state not in [12,13,14,15])))
            if action != "Down" :
                transitions[action].append((1/3, state if state in [0,1,2,3] else state-4, int(state-4 == 15 and state not in [0,1,2,3]), bool(state-4 in [5,7,11,12,15] and state not in [0,1,2,3])))
            if action != "Left" :
                transitions[action].append((1/3, state if state in [3,7,11,15] else state+1, int(state+1 == 15 and state not in [3,7,11,15]), bool(state+1 in [5,7,11,12,15] and state not in [3,7,11,15]))) 
            if action != "Right" :
                transitions[action].append((1/3, state if state in [0,4,8,12] else state-1, int(state-1 == 15 and state not in [0,4,8,12]), bool(state-1 in [5,7,11,12,15] and state not in [0,4,8,12])))           
        else :
            transitions[action].append((1, state, 0, True))
    fl_mdp[state] = transitions

You might need to do some stuff manually, but make sure to automate most of it.

You can check your implementation of the FL environment by comparing it with the one in OpenAI Gym.
OpenAI Gym basically contains implmentations of common RL MDPs so that we can directly train our agents on test environments. 

In [36]:
import gymnasium as gym
env = gym.make('FrozenLake-v1')
P = env.unwrapped.P

Since the imported MDP is also just a dictionary, we can just print it.

In [37]:
# using the pretty print module

import pprint
pprint.pprint(P)

{0: {0: [(0.33333333333333337, 0, 0, False),
         (0.3333333333333333, 0, 0, False),
         (0.33333333333333337, 4, 0, False)],
     1: [(0.33333333333333337, 0, 0, False),
         (0.3333333333333333, 4, 0, False),
         (0.33333333333333337, 1, 0, False)],
     2: [(0.33333333333333337, 4, 0, False),
         (0.3333333333333333, 1, 0, False),
         (0.33333333333333337, 0, 0, False)],
     3: [(0.33333333333333337, 1, 0, False),
         (0.3333333333333333, 0, 0, False),
         (0.33333333333333337, 0, 0, False)]},
 1: {0: [(0.33333333333333337, 1, 0, False),
         (0.3333333333333333, 0, 0, False),
         (0.33333333333333337, 5, 0, True)],
     1: [(0.33333333333333337, 0, 0, False),
         (0.3333333333333333, 5, 0, True),
         (0.33333333333333337, 2, 0, False)],
     2: [(0.33333333333333337, 5, 0, True),
         (0.3333333333333333, 2, 0, False),
         (0.33333333333333337, 1, 0, False)],
     3: [(0.33333333333333337, 2, 0, False),
         (0.

In [38]:
pprint.pprint(fl_mdp)

{0: {'Down': [(0.3333333333333333, 4, 0, False),
              (0.3333333333333333, 1, 0, False),
              (0.3333333333333333, 0, 0, False)],
     'Left': [(0.3333333333333333, 4, 0, False),
              (0.3333333333333333, 0, 0, False),
              (0.3333333333333333, 0, 0, False)],
     'Right': [(0.3333333333333333, 4, 0, False),
               (0.3333333333333333, 0, 0, False),
               (0.3333333333333333, 1, 0, False)],
     'Up': [(0.3333333333333333, 0, 0, False),
            (0.3333333333333333, 1, 0, False),
            (0.3333333333333333, 0, 0, False)]},
 1: {'Down': [(0.3333333333333333, 5, 0, True),
              (0.3333333333333333, 2, 0, False),
              (0.3333333333333333, 0, 0, False)],
     'Left': [(0.3333333333333333, 5, 0, True),
              (0.3333333333333333, 1, 0, False),
              (0.3333333333333333, 0, 0, False)],
     'Right': [(0.3333333333333333, 5, 0, True),
               (0.3333333333333333, 1, 0, False),
               (0

## Environment 3 - Key Door Game

Now we will implement a slightly different version of frozen lake without the stochasticity but with the added constraint of a key.

So now we have a 4x4 grid in which some cells are holes(and so terminal) and we have a goal state(keep it as 15) but this time someone locked the goal state with a door :(. Without the key we CANNOT ENTER the goal state.

Luckily for us, that guy was very clumsy and dropped the key in tile 3!. So now make a MDP to represent the process of us escaping this grid world.

Remember that the state in a MDP isnt just the position/location we are in. it should represent our whole game situation. (For example I can either be at tile 14 with the key or at tile 14 without key and these two situations are different and therfore must be represented as **different** states!)

To summarise
- state state is 0
- door/goal state is 15
- holes at 5, 7, 11, 12
- key at 3
- The key is picked up automatically upon entering tile 3
- can only enter tile 15 when key is obtained(otherwise its like a wall)
- holes & goal are terminal
- reward of 1 when reaching tile 15 wiht the key
- Attempting to enter tile 15 when we DONT have the key should be treated like hitting a wall and bouncing back to the original state where we took that action

In [39]:
# I hope these variables are named correctly
# Zoom out the screen for better readability(Sorry it was my mistake)
up_bou = [0, 1, 2, 3]
low_bou = [12, 13, 14, 15]
right_bou = [3, 7, 11, 15]
left_bou = [0, 4, 8, 12]
holes = [5, 7, 11, 12]
goal = 15
key_id = 3
action_space = ["Up", "Down", "Right", "Left"]


def transitions_dict (id, has_key) :
    dict = {} # dictionary I am going to return to the state (id, has_key)
    if id in holes or id == goal:
        # Current id is terminal
        for action in action_space :
            dict[action] = [(1, (id, has_key), 0, True)]
    else :
        # Now current id is non terminal
        # for every move either we have to travel to other id or return back because of 
        """ 
            first case : we are at the boundary and tried to move out 
            second case : we tried to enter goal without key
        """
        """ 
            first case of returning back is taken care using if-else statements
            second case of returning back is taken care directly in the assigning statement
            (for examle : using id-4 = goal and dont have key in Up action )
        """
        # Up
        if id not in up_bou :
            # the tuple i am assigning to Up action :
            # (probability=1,(______________next_id_____________________________,______next_has_key_______) , _______reward________________,___ did it reach terminal?______)
            dict["Up"] = [(1, (id if id-4 == goal and has_key == False else id-4 , has_key or id-4 == key_id), int(id-4 == goal and has_key), id-4 in holes or id-4 == goal and has_key)]
        else :
            dict["Up"] = [(1, (id, has_key), 0, False)]
        # Down
        if id not in low_bou :
            dict["Down"] = [(1, (id if id+4 == goal and has_key == False else id+4 , has_key or id+4 == key_id), int(id+4 == goal and has_key), id+4 in holes or id+4 == goal and has_key)]
        else :
            dict["Down"] = [(1, (id, has_key), 0, False)]
        # Right
        if id not in right_bou :
            dict["Right"] = [(1, (id if id+1 == goal and has_key == False else id+1 , has_key or id+1 == key_id), int(id+1 == goal and has_key), id+1 in holes or id+1 == goal and has_key)]
        else :
            dict["Right"] = [(1, (id, has_key), 0, False)]
        # Left
        if id not in left_bou :
            dict["Left"] = [(1, (id if id-1 == goal and has_key == False else id-1 , has_key or id-1 == key_id), int(id-1 == goal and has_key), id-1 in holes or id-1 == goal and has_key)]
        else :
            dict["Left"] = [(1, (id, has_key), 0, False)]
    return dict


gw_map = {
    (id, has_key) : transitions_dict(id, has_key) for id in range (0,16) for has_key in [True, False]
}

In [40]:
pprint.pprint(gw_map)

{(0, False): {'Down': [(1, (4, False), 0, False)],
              'Left': [(1, (0, False), 0, False)],
              'Right': [(1, (1, False), 0, False)],
              'Up': [(1, (0, False), 0, False)]},
 (0, True): {'Down': [(1, (4, True), 0, False)],
             'Left': [(1, (0, True), 0, False)],
             'Right': [(1, (1, True), 0, False)],
             'Up': [(1, (0, True), 0, False)]},
 (1, False): {'Down': [(1, (5, False), 0, True)],
              'Left': [(1, (0, False), 0, False)],
              'Right': [(1, (2, False), 0, False)],
              'Up': [(1, (1, False), 0, False)]},
 (1, True): {'Down': [(1, (5, True), 0, True)],
             'Left': [(1, (0, True), 0, False)],
             'Right': [(1, (2, True), 0, False)],
             'Up': [(1, (1, True), 0, False)]},
 (2, False): {'Down': [(1, (6, False), 0, False)],
              'Left': [(1, (1, False), 0, False)],
              'Right': [(1, (3, True), 0, False)],
              'Up': [(1, (2, False), 0, False)]},

The exact way you represent each state and each action is up to you! But... do atleast write down in a comment or something how you represent the states and action in the MDP. So i don't have to spend time deciphering.

Note: For additional challenges
- Make this grid-world also stochastic like Frozen Lake
- Make a function that allows you to input in the locations of the holes, the door(goal state) and the key and using that it will generate and return the MDP

In [44]:
# Below I wrote the solution to only one of the additional challenges 
# I didn't do first challenge because it was too big😢

In [ ]:
up_bou = [0, 1, 2, 3]
low_bou = [12, 13, 14, 15]
right_bou = [3, 7, 11, 15]
left_bou = [0, 4, 8, 12]
action_space = ["Up", "Down", "Right", "Left"]

def transitions_dict_cpy (id, has_key, holes, goal, key_id) :
    dict = {}
    if id in holes or id == goal:
        for action in action_space :
            dict[action] = [(1, (id, has_key), 0, True)]
    else :
        if id not in up_bou :
            dict["Up"] = [(1, (id if id-4 == goal and has_key == False else id-4 , has_key or id-4 == key_id), int(id-4 == goal and has_key), id-4 in holes or id-4 == goal and has_key)]
        else :
            dict["Up"] = [(1, (id, has_key), 0, False)]
        if id not in low_bou :
            dict["Down"] = [(1, (id if id+4 == goal and has_key == False else id+4 , has_key or id+4 == key_id), int(id+4 == goal and has_key), id+4 in holes or id+4 == goal and has_key)]
        else :
            dict["Down"] = [(1, (id, has_key), 0, False)]
        if id not in right_bou :
            dict["Right"] = [(1, (id if id+1 == goal and has_key == False else id+1 , has_key or id+1 == key_id), int(id+1 == goal and has_key), id+1 in holes or id+1 == goal and has_key)]
        else :
            dict["Right"] = [(1, (id, has_key), 0, False)]
        if id not in left_bou :
            dict["Left"] = [(1, (id if id-1 == goal and has_key == False else id-1 , has_key or id-1 == key_id), int(id-1 == goal and has_key), id-1 in holes or id-1 == goal and has_key)]
        else :
            dict["Left"] = [(1, (id, has_key), 0, False)]
    return dict

def key_door () :
    holes = [int(x) for x in input("Enter the cell id's of holes seperated by spaces :").split()]
    goal = int(input("Enter the cell id of the goal :"))
    key_id = int(input("Enter the id of the cell where the key is there :"))
    gw_map_return = {
        (id, has_key) : transitions_dict_cpy(id, has_key, holes, goal, key_id) for id in range (0,16) for has_key in [True, False]
    }
    return gw_map_return

In [42]:
gw_map_new = key_door()
# I gave here inputs as previous situation
# tried to check is gw_map_new is same as gw_map
print(gw_map_new == gw_map) 


True


## Environment 4 - Lights Out

**Lights out** is a deterministic puzzle game which consists of a 5x5 grid of lights which can either be on or off.

When the game begins a random set of these lights will be switched on. Pressing any light will toggle it (toggle means if the light was on then it will become off and if it was off then it will become on) and it will also toggle all of its neighbours.

![image of the game](./images/LightsOutIllustration.png)

Note: so any non-edge and non-corner cell action will toggle 5 lights, an edge cell action would toggle 4 lights and a corner cell action only toggles 3 lights.


The goal of the game is to find a sequence of moves to switch off all the lights. Now i'm not gonna make you solve this instead we are going to represent this as a deterministic MDP, where 
- the states should represent the game grid layout 
- the actions are the cells you wish to toggle
- Give a reward of 1 when solved 
- the ONLY terminal state is the fully solved state

Since 5x5 is pretty big we will restrict ourselves to a **4x4** grid and represent it as an MDP instead

In [60]:
# Zoom out the screen for better readability(Sorry it was my mistake)
"""
I represented every state with a binary string of 17 digits starting with 1

the 4*4 grid cells have the numbering as shown below
   1    2     3    4
   5    6     7    8
   9    10    11   12
   13   14    15   16

string[i] = 0 means : cell with id i is on
          = 1 means : cell with id i is off

action is represented by the id of the cell we are going to select

toggle function takes a string(current state) and list(of id's we have to toggle)
as inputs and returns a string(new state) 

the terminal state is string with all 1's(all lights are off)

"""
state_space=[]

up_edge = [2, 3]
low_edge = [14, 15]
right_edge = [8, 12]
left_edge = [5, 9]
corners = [1, 4, 13, 16]

for i in range(2**16, 2**17 - 1) :
    state_space.append(str(bin(i)))

def toggle(strin, list) :
    str_cpy = "1"
    for i in range(1, 17) :
        if i in list :
            if strin[i] :
                str_cpy = str_cpy + "0"
            else :
                str_cpy = str_cpy + "1"
        else :
            if strin[i] :
                str_cpy = str_cpy + "1"
            else :
                str_cpy = str_cpy + "0"
    return str_cpy

def transition_dict (state) :
    dict = {}
    for i in range(1, 17) :
        if int(state, base= 2) == 2**17 -1 :
            dict[i] = [(1, state, 0, True)]
        else :
            done = False
            list =[]
            if i in up_edge :
                list = [i-1, i, i+1, i+4]
            elif i in low_edge :
                list = [i-4, i-1, i, i+1]
            elif i in left_edge :
                list = [i-4, i, i+1, i+4]
            elif i in right_edge :
                list = [i-4, i-1, i, i+4]
            elif i in corners :
                if i == 1 :
                    list = [1, 2, 5]
                if i == 4 :
                    list = [3, 4, 8]
                if i == 13 :
                    list = [9, 13, 14]
                if i == 16 :
                    list = [12, 15, 16]
            else :
                list = [i-4, i-1, i, i+1, i+4]    
            new_state = toggle(state, list)
            if int(new_state, base= 2) == 2**17-1 :
                done = True
            dict[i] = [(1, new_state, int(done), done)]
    return dict

lo_map = {
    state : transition_dict(state)  for state in state_space
}

The exact way you represent each state and each action is up to you! But... do atleast write down in a comment or something how you represent the states and action in the MDP. So i don't have to spend time deciphering.

Note: if you want you can also try to represent the 5x5 one but this is optional